# 473 Matchsticks to Square

这题我写出了一个……爆炸性的 TLE……

她的时间复杂度…是…O(4 ^ n)…

思路是 DFS 地往四条边上放火柴。

上过算法课的人写出这种算法，当撞墙自尽。

事实上他走到 [16,8,8,8,5,1,16,3,11,1,11,12,20,6,6] 这个 Case 就爆炸了…

In [ ]:
#!/usr/bin/env python

from functools import lru_cache


class Solution:
    def makesquare(self, nums: List[int]) -> bool:
        if nums == []:
            return False

        sumup = sum(nums)
        numcnts = len(nums)

        if sumup % 4 != 0:
            return False

        target = sumup // 4

        if max(nums) > target:
            return False

        @lru_cache(maxsize=1000)
        def dfs(idx: int, l1: int = 0, l2: int = 0, l3: int = 0, l4: int = 0) -> bool:
            if idx == numcnts:
                return l1 == l2 and l2 == l3 and l3 == l4
            return dfs(idx + 1, l1 + nums[idx], l2, l3, l4) or dfs(idx + 1, l1, l2 + nums[idx], l3, l4) or dfs(idx + 1, l1, l2, l3 + nums[idx], l4) or dfs(idx + 1, l1, l2, l3, l4 + nums[idx])

        return dfs(0)


好吧，咱们还是好好看看有没有什么可优化的点。

首先，lru_cache 几乎不可能命中，这时候加这个只是浪费空间而已。而且，我们没必要拷贝大量的 l1 - l4，而是可以通过一个全局值来存放。
因为每一个 dfs 都可以在需要的时候撤销自己带来的影响（放上去的那一根火柴）。

另外，假如放上一根火柴会使得长度超限，那就不要 call 这个 dfs。

经过这些比较浅显的优化之后，还是会爆炸。

这里有个 Trick：我们先做逆向排序，尽可能先放长木棍。

首先，这个改变不会影响正确性（显然），能组成的还是能组成。

但是，trying a longer matchstick first will get to negative conclusion earlier.

先尝试比较长的火柴，那么那些「不可能组合」暴露得就越快。一些边的长度会最快地超过平均水准，这会使得无法再放置木棍。